# Player Dataprep

In [50]:
from nba_api.stats.endpoints import leaguegamelog
import pandas as pd

YEAR = 2023

COUNTER = 0
DIRECTION = "ASC"
LEAGUE = "00"
PLAYER_OR_TEAM = "P"
SEASON_TYPE = "Regular Season"
SORTER = "DATE"

def fetch_season_game_logs(season):
    # Create LeagueGameLog object instance for the season
    gamelog = leaguegamelog.LeagueGameLog(
        counter=COUNTER,
        direction=DIRECTION,
        league_id=LEAGUE,
        player_or_team_abbreviation=PLAYER_OR_TEAM,
        season=season,
        season_type_all_star=SEASON_TYPE,
        sorter=SORTER
    )
    
    # Execute request and fetch data
    data = gamelog.get_data_frames()[0]
    
    # Filter columns and add Home/Away column
    columns_to_keep = [
       "GAME_ID", "TEAM_ID", "TEAM_NAME", "PLAYER_ID", "PLAYER_NAME", "MATCHUP", "WL", "GAME_DATE",
        "PTS", "OREB", "DREB", "AST", "STL", "BLK", "TOV", "FGM", "FGA", "FG_PCT",
        "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "MIN"
    ]
    filtered_data = data[columns_to_keep].copy()
    filtered_data['HOME/AWAY'] = filtered_data['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    
    return filtered_data

def fetch_multiple_seasons(start_year, end_year):
    all_seasons_data = []  # List to store data for all seasons
    
    # Loop through each season from start_year to end_year (inclusive)
    for year in range(start_year, end_year + 1):
        season = f"{year}-{str(year + 1)[-2:]}"  # Format season string (e.g., '2010-11')
        season_data = fetch_season_game_logs(season)
        all_seasons_data.append(season_data)
    
    # Concatenate all seasons' data into a single DataFrame
    combined_data = pd.concat(all_seasons_data, ignore_index=True)
    combined_data = combined_data.sort_values(by='GAME_ID', ascending=True)
    combined_data = combined_data.reset_index(drop=True)
    
    return combined_data

def filter_top_players(data):
    # Group by GAME_ID and TEAM_ID, then sort by MIN within each group and take the top 7 players
    top_players = data.groupby(['GAME_ID', 'TEAM_ID']).apply(lambda x: x.nlargest(7, 'MIN')).reset_index(drop=True)
    return top_players

def ensure_minimum_players(data, min_players=5):
    # Ensure each game has at least `min_players` players
    def fill_missing_players(group):
        if len(group) < min_players:
            missing_count = min_players - len(group)
            for _ in range(missing_count):
                # Add a row with missing player information and zeroed stats
                missing_row = pd.Series({
                    'GAME_ID': group['GAME_ID'].iloc[0],
                    'TEAM_ID': group['TEAM_ID'].iloc[0],
                    'TEAM_NAME': group['TEAM_NAME'].iloc[0],
                    'PLAYER_ID': None,
                    'PLAYER_NAME': 'N/A',
                    'MATCHUP': group['MATCHUP'].iloc[0],
                    'WL': group['WL'].iloc[0],
                    'GAME_DATE': group['GAME_DATE'].iloc[0],
                    'PTS': 0, 'OREB': 0, 'DREB': 0, 'AST': 0, 'STL': 0, 'BLK': 0, 'TOV': 0,
                    'FGM': 0, 'FGA': 0, 'FG_PCT': 0, 'FG3M': 0, 'FG3A': 0, 'FG3_PCT': 0,
                    'FTM': 0, 'FTA': 0, 'FT_PCT': 0, 'MIN': 0, 'HOME/AWAY': group['HOME/AWAY'].iloc[0]
                })
                group = group.append(missing_row, ignore_index=True)
        return group
    
    return data.groupby(['GAME_ID', 'TEAM_ID']).apply(fill_missing_players).reset_index(drop=True)

def pivot_player_stats(data):
    data = filter_top_players(data)
    data = ensure_minimum_players(data)
    
    # Separate home and away teams
    home_data = data[data['HOME/AWAY'] == 1].copy()
    away_data = data[data['HOME/AWAY'] == 0].copy()

    print(home_data)
    
    # Rank players within each game by playtime to label columns
    home_data['RANK'] = home_data.groupby('GAME_ID')['MIN'].rank("dense", ascending=False).astype(int)
    away_data['RANK'] = away_data.groupby('GAME_ID')['MIN'].rank("dense", ascending=False).astype(int)
    
    # Pivot the data
    home_pivot = home_data.pivot_table(
        index=['GAME_ID', 'GAME_DATE'],
        columns='RANK',
        values=['PLAYER_NAME', 'PTS', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'FGM', 'FGA', 'FG_PCT', 
                'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'MIN'],
        aggfunc='first'
    )
    away_pivot = away_data.pivot_table(
        index=['GAME_ID', 'GAME_DATE'],
        columns='RANK',
        values=['PLAYER_NAME', 'PTS', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'FGM', 'FGA', 'FG_PCT', 
                'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'MIN'],
        aggfunc='first'
    )
    
    # Flatten the MultiIndex columns and label them appropriately
    home_pivot.columns = [f'HomeTeamPlayer{rank}_{stat}' for stat, rank in home_pivot.columns]
    away_pivot.columns = [f'AwayTeamPlayer{rank}_{stat}' for stat, rank in away_pivot.columns]
    
    # Reset index
    home_pivot = home_pivot.reset_index()
    away_pivot = away_pivot.reset_index()
    
    # Debug: print the unique GAME_IDs
    print("Unique GAME_IDs in home_pivot:", home_pivot['GAME_ID'].unique())
    print("Unique GAME_IDs in away_pivot:", away_pivot['GAME_ID'].unique())
    
    # Merge home and away data back together
    merged_data = pd.merge(home_pivot, away_pivot, on=['GAME_ID', 'GAME_DATE'], suffixes=('_home', '_away'))
    
    return merged_data

# Fetch player game logs for the 2023 season
player_game_logs = fetch_multiple_seasons(2023, 2023)

# Pivot the data to get one row per game with the top 7 players from each team
pivoted_player_stats = pivot_player_stats(player_game_logs)

print("Pivoted player stats shape:", pivoted_player_stats.shape)
pivoted_player_stats


Top players filtered data shape: (17220, 26)
Home data shape: (8610, 26)
Away data shape: (8610, 26)
Unique GAME_IDs in home_pivot: ['0022300001' '0022300002' '0022300003' ... '0022301228' '0022301229'
 '0022301230']
Unique GAME_IDs in away_pivot: ['0022300001' '0022300002' '0022300003' ... '0022301228' '0022301229'
 '0022301230']
Pivoted player stats shape: (1230, 254)


,GAME_ID,GAME_DATE,HomeTeamPlayer1_AST,HomeTeamPlayer2_AST,HomeTeamPlayer3_AST,HomeTeamPlayer4_AST,HomeTeamPlayer5_AST,HomeTeamPlayer6_AST,HomeTeamPlayer7_AST,HomeTeamPlayer1_BLK,...,AwayTeamPlayer5_STL,AwayTeamPlayer6_STL,AwayTeamPlayer7_STL,AwayTeamPlayer1_TOV,AwayTeamPlayer2_TOV,AwayTeamPlayer3_TOV,AwayTeamPlayer4_TOV,AwayTeamPlayer5_TOV,AwayTeamPlayer6_TOV,AwayTeamPlayer7_TOV
0,0022300001,2023-11-03,2.0,13.0,0.0,2.0,0.0,5.0,1.0,0.0,...,1.0,0.0,2.0,3.0,2.0,4.0,1.0,1.0,0.0,1.0
1,0022300002,2023-11-03,4.0,2.0,3.0,4.0,NaN,NaN,NaN,1.0,...,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,1.0,1.0
2,0022300003,2023-11-03,9.0,2.0,3.0,5.0,3.0,NaN,NaN,0.0,...,0.0,1.0,NaN,5.0,2.0,1.0,2.0,2.0,2.0,NaN
3,0022300004,2023-11-03,5.0,3.0,2.0,4.0,2.0,4.0,1.0,1.0,...,NaN,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN
4,0022300005,2023-11-03,1.0,8.0,5.0,4.0,0.0,1.0,0.0,0.0,...,0.0,2.0,NaN,1.0,3.0,3.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,0022301226,2023-12-08,8.0,0.0,3.0,2.0,1.0,NaN,NaN,0.0,...,1.0,0.0,1.0,5.0,1.0,1.0,1.0,2.0,1.0,0.0
1226,0022301227,2023-12-08,5.0,5.0,3.0,1.0,4.0,1.0,NaN,0.0,...,0.0,0.0,NaN,1.0,0.0,1.0,1.0,0.0,1.0,NaN
1227,0022301228,2023-12-08,3.0,3.0,3.0,1.0,1.0,6.0,NaN,1.0,...,0.0,3.0,0.0,4.0,2.0,0.0,1.0,2.0,1.0,2.0
1228,0022301229,2023-12-07,7.0,2.0,2.0,1.0,4.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0


In [18]:
# Group by GAME_ID and count the number of unique PLAYER_ID entries for each game
player_count_per_game = player_game_logs.groupby('GAME_ID')['PLAYER_ID'].nunique().reset_index()

# Rename columns for clarity
player_count_per_game.columns = ['GAME_ID', 'PLAYER_COUNT']

# Display the result
print(player_count_per_game.head())

# Summary statistics
summary_stats = player_count_per_game['PLAYER_COUNT'].describe()
print(summary_stats)


      GAME_ID  PLAYER_COUNT
0  0022300001            19
1  0022300002            19
2  0022300003            23
3  0022300004            18
4  0022300005            20
count    1230.000000
mean       21.464228
std         2.885096
min        16.000000
25%        19.000000
50%        21.000000
75%        24.000000
max        30.000000
Name: PLAYER_COUNT, dtype: float64
